In [1]:
# import all packages

import numpy as np

In [2]:
data = np.loadtxt('DMDdata200.txt', delimiter=',')

In [3]:
x1 = data[0:,1:-1]
x2 = data[0:,2:]

In [4]:
data_shape = (x1.shape[1], x1.shape[0])
percent_Trunc = 0.99
frame_rate = 1/0.05
# r = 50

In [5]:
[U, S, V] = np.linalg.svd(x1, full_matrices=False, compute_uv=True, hermitian=False)

In [6]:
menergy = 0.0
for i in range(0, np.size(S)):
    menergy += S[i]/np.sum(S)
    if menergy > percent_Trunc:
        break
    r = i
    
menergy, r

(0.9907489736735519, 17)

In [8]:
Atilde = U[:, : r].conj().T @ x2 @ V[:r, :].conj().T * np.reciprocal(S[: r])
[eivVal,eivVec] = np.linalg.eig(Atilde)

In [9]:
Atilde.shape

(17, 17)

In [11]:
phi= x2 @ V[:r, :].conj().T @ np.diag(np.reciprocal(S[: r])) @ eivVec
b = np.linalg.lstsq(phi, x1[:, 0])
vandermodeV = np.vander(eivVal)
eivarray = np.diag(eivVal)

C:\Users\sumuk\AppData\Local\Temp\ipykernel_10076\973257988.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b = np.linalg.lstsq(phi, x1[:, 0])


In [13]:
np.abs(b[0]).real.sum()

17.927700448533958

In [ ]:
# DMD

# [U, S, V] = np.linalg.svd(y1, full_matrices=False, compute_uv=True, hermitian=False)
# Atilde = U[:, : r].conj().T @ y2 @ V[:r, :].conj().T * np.reciprocal(S[: r])
# U[:, : r].conj().T @ np.dot(x2.T, x2) @ V[: r, :].conj().T * np.reciprocal(S[: r])
# Atilde = np.dot(Atilde.T, Atilde)
# [eivVal,eivVec] = np.linalg.eig(Atilde)
# eivVal_sorted = -np.sort(-eivVal)
# eivarray = np.diag(eivVal)
# psi = y2 @ V[:r, :].conj().T @ np.diag(np.reciprocal(S[: r])) @ eivVec # / np.tile(np.transpose(eivarray),(np.size(y1,1),1)) 
# d = X1, y1[:,0] # scales
# d = np.linalg.solve(X1.T.dot(X1), X1.T.dot(y1))
# amplitude = np.abs(d)
# X=X1*np.diag(d) # scaled DMD modes
# sp_modes = y2 @ V[:r, :].conj().T @ np.diag(np.reciprocal(S[: r])) @ eivVec
# A = psi @ np.diag(eivVal) #@ np.linalg.pinv(X1)
# end of DMD

In [ ]:
# menergy = 0.0 #eivVal_sorted.shape
# for i in range(0, np.size(eivVal_sorted)):
#     # print(i)
#     menergy += eivVal_sorted[i]/np.sum(eivVal_sorted)

# menergy

In [ ]:
# modeNorm = np.zeros(r)
# for i in range(0, r-1):
#     modeNorm[i] = np.linalg.norm(X[:,i]) # norm of each mode

# mag = np.absolute(eivarray)
# weigh = mag[1:-1]/np.sum(mag)
# delta_t = 1/frame_rate   
# omega = np.log(eivarray)/delta_t
# growth = np.real(omega)
# frequency = np.imag(omega)/(2*np.pi)

In [ ]:
# # #

# freq1 = np.where(frequency>=0)     # eliminating negative frequencies
# amplitude1 = amplitude[freq1]  # eliminating amplitudes corresponding to -ve frequencies
# d1 = d[freq1]    # similar operations below for different parameters
# omega1 = omega[freq1]
# growth1 = growth[freq1]
# # weigh1 = weigh[freq1]
# # modeNorm1 = modeNorm[:,freq1]
# # frequency1 = frequency[freq1]
# # Xplus = X[:freq1]